In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import importlib
import copy
import argparse
import measures
from torchvision import transforms, datasets
from main import *


In [4]:
dataset = load_data('train','CIFAR10', 'datasets', 3)

Files already downloaded and verified


In [9]:
dataset.__getitem__(0)[0].shape

torch.Size([3, 32, 32])

In [5]:
from models.vgg import Network
model = Network(1,10)

In [14]:
update_json_file(1,{1:2,'marie':'jesus'})

In [6]:
from measures import compute_singular_values
from operator_norm import *

In [7]:
kernel = model.features[0].weight.data

In [10]:
singular_values(kernel,(32,32)).max()

3.898205881579876

In [28]:
inp_shape = (64,64)
clip_to = 1
transform_coeff = np.fft.fft2(kernel, inp_shape, axes=[0, 1])

  # now, for each transform coefficient, compute the singular values of the
  # matrix obtained by selecting that coefficient for
  # input-channel/output-channel pairs
U, D, V = np.linalg.svd(transform_coeff, compute_uv=True, full_matrices=False)
D_clipped = np.minimum(D, clip_to)

if kernel.shape[2] > kernel.shape[3]:
    clipped_transform_coeff = np.matmul(U, D_clipped[..., None] * V)
else:
    clipped_transform_coeff = np.matmul(U * D_clipped[..., None, :], V)
clipped_filter = np.fft.ifft2(clipped_transform_coeff, axes=[0, 1]).real
args = [range(d) for d in kernel.shape]
print(args)
clipped_filter[np.ix_(*args)]

[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]


array([[[[-0.02809479,  0.08737295, -0.11070255],
         [-0.09037959,  0.12573889,  0.0325132 ],
         [-0.03128234, -0.00625614, -0.09087451]]],


       [[[ 0.02106376,  0.07682895,  0.04837548],
         [-0.0448083 ,  0.10111518, -0.00496176],
         [-0.11493986, -0.13388247, -0.08087595]]],


       [[[-0.05669635,  0.0045864 , -0.09225942],
         [ 0.05648958,  0.04360816,  0.07874691],
         [-0.13331759,  0.04864646, -0.0795561 ]]],


       [[[-0.05449392,  0.09757533, -0.05140037],
         [ 0.11393849,  0.08116613, -0.043236  ],
         [ 0.07108502,  0.04575966,  0.04617897]]],


       [[[ 0.0121054 ,  0.06150948, -0.0073066 ],
         [ 0.00472739,  0.02261022, -0.01629708],
         [ 0.07625816, -0.05201912,  0.02302471]]],


       [[[ 0.02810456,  0.0472548 , -0.05558115],
         [ 0.04194324,  0.1195994 ,  0.09015186],
         [-0.04317373, -0.0581671 ,  0.05483778]]],


       [[[ 0.08042092,  0.0529395 , -0.01894247],
         [ 0.12792008, -0.

In [15]:
kernel.shape

torch.Size([64, 1, 3, 3])

In [16]:
D.shape

(32, 32, 3)

In [12]:
Clip_OperatorNorm_NP(kernel,(32,32), 3 )

[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]


ValueError: cannot reshape array of size 9216 into shape (64,1,3,3)

In [86]:
project_model(model, (34,34),1)

[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\nevor\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nevor\AppData\Local\Temp\ipykernel_3432\3264386094.py", line 1, in <module>
    project_model(model, (34,34),1)
  File "c:\Users\nevor\OneDrive\Documents\Cours\TDL\PROJET_FINAL\operator_norm.py", line 105, in project_model
    clipped_weights = Clip_OperatorNorm_NP(weights, inp_shape, clip_to)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nevor\OneDrive\Documents\Cours\TDL\PROJET_FINAL\operator_norm.py", line 97, in Clip_OperatorNorm_NP
    return clipped_filter.reshape(filter.shape)#[np.ix_(*args)]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: cannot reshape array of size 10404 into shape (64,1,3,3)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\User

In [76]:
def Clip_OperatorNorm_NP(filter, inp_shape, clip_to):
  # compute the singular values using FFT
  # first compute the transforms for each pair of input and output channels
  transform_coeff = np.fft.fft2(filter, inp_shape, axes=[0, 1])

  # now, for each transform coefficient, compute the singular values of the
  # matrix obtained by selecting that coefficient for
  # input-channel/output-channel pairs
  U, D, V = np.linalg.svd(transform_coeff, compute_uv=True, full_matrices=False)
  D_clipped = np.minimum(D, clip_to)
  if filter.shape[2] > filter.shape[3]:
    clipped_transform_coeff = np.matmul(U, D_clipped[..., None] * V)
  else:
    clipped_transform_coeff = np.matmul(U * D_clipped[..., None, :], V)
  clipped_filter = np.fft.ifft2(clipped_transform_coeff, axes=[0, 1]).real
  args = [range(d) for d in filter.shape]
  print(args)
  print(clipped_filter.shape)

  return clipped_filter[np.ix_(*args)]


def project_model(model, inp_shape, clip_to):
    with torch.no_grad():
        for i,layer in enumerate(model.features):
            if isinstance(layer, nn.Conv2d):
                weights = layer.weight.data
                clipped_weights = Clip_OperatorNorm_NP(weights, inp_shape, clip_to)
                model.features[i].weight.data = torch.tensor(clipped_weights)
            elif isinstance(layer, nn.Linear):
                weights = layer.weight.data
                clipped_weights = Clip_OperatorNorm_NP(weights, weights[-2:].shape, clip_to)
                model.features[i].weight.data = torch.tensor(clipped_weights)
            
        

In [80]:
weights.shape

torch.Size([64, 1, 3, 3])

In [82]:
with torch.no_grad():
        for i,layer in enumerate(model.features):
            if isinstance(layer, nn.Conv2d):
                weights = layer.weight.data
                clipped_weights = Clip_OperatorNorm_NP(weights, (32,32), 1)
                model.features[i].weight.data = torch.tensor(clipped_weights)
            elif isinstance(layer, nn.Linear):
                weights = layer.weight.data
                clipped_weights = Clip_OperatorNorm_NP(weights, weights[-2:].shape, 1)
                model.features[i].weight.data = torch.tensor(clipped_weights)
            
        

[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]
(32, 32, 3, 3)


IndexError: index 32 is out of bounds for axis 0 with size 32

In [77]:
project_model(model2,(32,32),1)


[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]
(32, 32, 3, 3)


IndexError: index 32 is out of bounds for axis 0 with size 32

In [71]:
model1, model2 = Network(1,6),Network(1,6)
print(compute_singular_values(model1, model2,(32,32)))
project_model(model1,(100,100),1)
print('AAAAAA')
project_model(model2,(32,32),1)
print(compute_singular_values(model1, model2,(32,32)))


(902.6261525200096, 4947520)
[range(0, 64), range(0, 1), range(0, 3), range(0, 3)]
[range(0, 64), range(0, 64), range(0, 3), range(0, 3)]
[range(0, 128), range(0, 64), range(0, 3), range(0, 3)]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\nevor\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\nevor\AppData\Local\Temp\ipykernel_3432\4089121324.py", line 3, in <module>
    project_model(model1,(100,100),1)
  File "c:\Users\nevor\OneDrive\Documents\Cours\TDL\PROJET_FINAL\operator_norm.py", line 105, in project_model
    clipped_weights = Clip_OperatorNorm_NP(weights, inp_shape, clip_to)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nevor\OneDrive\Documents\Cours\TDL\PROJET_FINAL\operator_norm.py", line 97, in Clip_OperatorNorm_NP
    return clipped_filter[np.ix_(*args)]
           ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
IndexError: index 100 is out of bounds for axis 0 with size 100

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\nevor\anaconda3\Lib\site-packag

In [14]:
np.prod(torch.tensor([[1,2],[1,2]]).to('cuda').shape)

4

In [55]:
for layer in model.features:
            if isinstance(layer, nn.Conv2d):
                    break

In [57]:
layer.weight.data

tensor([[[[-0., -0., 0.],
          [0., -0., -0.],
          [0., 0., 0.]]],


        [[[0., -0., 0.],
          [-0., 0., -0.],
          [-0., -0., -0.]]],


        [[[-0., -0., 0.],
          [0., -0., 0.],
          [-0., 0., 0.]]],


        [[[0., 0., -0.],
          [0., 0., -0.],
          [-0., -0., 0.]]],


        [[[-0., -0., 0.],
          [0., 0., -0.],
          [-0., 0., -0.]]],


        [[[0., 0., -0.],
          [-0., 0., 0.],
          [0., 0., -0.]]],


        [[[-0., 0., 0.],
          [0., -0., 0.],
          [-0., 0., -0.]]],


        [[[-0., 0., -0.],
          [-0., -0., 0.],
          [-0., -0., 0.]]],


        [[[-0., -0., -0.],
          [-0., -0., -0.],
          [0., 0., 0.]]],


        [[[-0., 0., -0.],
          [0., 0., -0.],
          [0., 0., -0.]]],


        [[[-0., -0., -0.],
          [-0., -0., 0.],
          [-0., -0., -0.]]],


        [[[0., -0., 0.],
          [-0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
        

In [33]:
model.state_dict()['classifier.4.weight'].shape

torch.Size([10, 512])

In [37]:
compute_singular_values(Network(1,10),Network(1,10), (32,32))

(923.9082232043822, 4949568)

In [53]:
model.features

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Dropout(p=0.5, inplace=False)
  (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Dropout(p=0.5, inplace=False)
  (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Dropout(p=0.5, inplace=False)
  (18): Conv2d(256, 512

In [52]:
model.features[0].weight.data

tensor([[[[-0., -0., 0.],
          [0., -0., -0.],
          [0., 0., 0.]]],


        [[[0., -0., 0.],
          [-0., 0., -0.],
          [-0., -0., -0.]]],


        [[[-0., -0., 0.],
          [0., -0., 0.],
          [-0., 0., 0.]]],


        [[[0., 0., -0.],
          [0., 0., -0.],
          [-0., -0., 0.]]],


        [[[-0., -0., 0.],
          [0., 0., -0.],
          [-0., 0., -0.]]],


        [[[0., 0., -0.],
          [-0., 0., 0.],
          [0., 0., -0.]]],


        [[[-0., 0., 0.],
          [0., -0., 0.],
          [-0., 0., -0.]]],


        [[[-0., 0., -0.],
          [-0., -0., 0.],
          [-0., -0., 0.]]],


        [[[-0., -0., -0.],
          [-0., -0., -0.],
          [0., 0., 0.]]],


        [[[-0., 0., -0.],
          [0., 0., -0.],
          [0., 0., -0.]]],


        [[[-0., -0., -0.],
          [-0., -0., 0.],
          [-0., -0., -0.]]],


        [[[0., -0., 0.],
          [-0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
        

In [22]:
model.state_dict()['features.0.weight']
 

tensor([[[[-9.1268e-02,  4.4824e-02,  4.0985e-02],
          [ 9.6848e-02,  1.8914e-01, -1.3705e-01],
          [ 3.2490e-01, -3.6996e-02, -2.5126e-01]]],


        [[[-1.2435e-01, -2.6728e-01, -1.1355e-01],
          [-9.8292e-02, -2.1810e-01, -3.0978e-01],
          [ 2.9060e-01,  6.7862e-02,  2.4920e-01]]],


        [[[-1.4066e-01,  2.7121e-01, -1.5174e-01],
          [ 8.4993e-02, -1.4346e-01,  8.1819e-03],
          [-2.7406e-01,  4.1625e-02, -7.5604e-02]]],


        [[[-7.5726e-02, -8.7678e-02,  3.2024e-01],
          [-2.5523e-01,  6.3943e-02, -2.2567e-02],
          [-4.5594e-02,  1.2936e-01, -9.0607e-02]]],


        [[[ 1.6405e-01,  2.0503e-01,  3.0147e-01],
          [ 2.8757e-01, -2.9974e-01, -4.8819e-02],
          [-2.8923e-02,  2.0651e-01, -1.3601e-01]]],


        [[[ 1.4354e-01,  2.6202e-02,  2.3262e-01],
          [ 2.9429e-01, -2.6217e-01,  1.4826e-01],
          [-2.4079e-01,  2.8717e-01, -1.7205e-02]]],


        [[[ 1.7444e-01,  2.1157e-01, -1.9170e-01],
       

In [17]:
x

'classifier.4.bias'